## Algorithm Design

### CNN

In [31]:
# This architectural CNN is from the mind of Deep Mind.
def cnn_model():
    # This model building approach uses the Keras Functional API.
    # The Keras Functional API can be found at: https://keras.io/guides/functional_api/
    image_framework = layers.Input(ATARI_IMAGE_SHAPE, name='image-shape-framework')
    # Normalize 0-255 image scale to within the constraints of 0 and 1.
    # This is done last minute because it is a heavy load to store normalized images, but they
    # can more easily be transformed into processing by the model.
    normalized_images = layers.Lambda(lambda x: x / 255.0, name='normalize-images')(image_framework)

    layer_one_hidden = layers.convolutional.Conv2D(
        LAYER_1_SIZE, LAYER_1_FILTER, strides=LAYER_1_STRIDES, activation=ACTIVATION_FUNCTION
    )(normalized_images)
    layer_two_hidden = layers.convolutional.Conv2D(
        LAYER_2_SIZE, LAYER_2_FILTER, strides=LAYER_2_STRIDES, activation=ACTIVATION_FUNCTION
    )(layer_one_hidden)
    # Flatten before connecting to move to 1D structure.
    flat_layer = layers.core.Flatten()(layer_two_hidden)
    # Dense layer, fully connected.
    fully_connected_layer = layers.Dense(256, activation=ACTIVATION_FUNCTION)(flat_layer)
    # Dense layer is fully connected.
    action_layers = layers.Dense(ACTION_OPTION_COUNT)(fully_connected_layer)
    # Action mask encodes.
    action_input = layers.Input((ACTION_OPTION_COUNT,), name='action-mask')
    # Multiply layer for each action using the encoded action mask.
    mult_res_layer = layers.Multiply(name='deep-q-cnn')([action_layers, action_input])

    model = Model(inputs=[image_framework, action_input], outputs=mult_res_layer)
    model.summary()
    # RMSProp is commonly used for minibatch learning.
    optimizer = RMSprop(lr=LEARNING_RATE, rho=0.95, epsilon=0.01)
    model.compile(optimizer, loss=huber_loss)
    return model

### Initialization Methods

In [56]:
def get_log_dir():
    curr_time = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    record_dir = "{}/openai-time-{}-logger".format(TRAIN_DIR, curr_time)

def init_file_writer_to_local_dir():
    return tf.summary.FileWriter(get_log_dir(), tf.get_default_graph())

def init_batch_matrix():
    return np.zeros((BATCH_SIZE, ATARI_IMAGE_SHAPE[0], ATARI_IMAGE_SHAPE[1], ATARI_IMAGE_SHAPE[2]))

def preprocess(observe):
    grayscale_imgs = rgb2gray(observe)
    shrink_imgs = resize(grayscale_imgs, (84, 84), mode='constant')
    processed_observe = np.uint8(shrink_imgs * 255)
    return processed_observe

def init_history(observe):
    # At start of game, there is no preceding frame.
    # So just copy initial states to make history.
    state = preprocess(observe)
    # state = preprocess(observe)
    history = np.stack((state, state, state, state), axis=2)
    history = np.reshape([history], (1, 84, 84, 4))
    return history

def init_test_config():
    env = gym.make('BreakoutDeterministic-v4')
    player_games = 0
    episode_number = 0
    epsilon = 0.001
    global_step = NUM_OBSERVABLE_STEPS + 1
    model = load_model(LOGS_FILE_PATH, custom_objects={'huber_loss': huber_loss})
    return (env, player_games, episode_number, epsilon, global_step, model)

def init_model_clone(model):
    # Copy model since actual model weights will get updated later TODO.  when?
    # Clone model using keras api function.
    model_clone = clone_model(model)
    # Clone model weights to new model separately
    model_clone.set_weights(model.get_weights())
    return model_clone

def init_config():
    env = gym.make('BreakoutDeterministic-v4')
    # Deque is imported from collections.  Set to a finite size.  New memory will overwrite old.
    memory = deque(maxlen=400000)
    # init
    epsilon = 1.0
    # Calc decay rate.
    epsilon_decay = EPSILON_DECAY
    total_steps = 0
    # Init at 0.
    player_game = 0
    return (env, memory, epsilon, epsilon_decay, total_steps, player_game)

def init_game_config():
    done = False
    dead = False
    game_step = 0
    game_score = 0
    game_lives = 5
    game_loss = 0.0
    return (done, dead, game_step, game_score, game_lives, game_loss)


### Helper Methods

In [57]:
# Action is random if it is an observed state or if by chance based on the epsilon threshold, it is.
# If action is not random it gets generated from the current model based on history data to this point.
# I select the best action from this result.
def get_action(history, epsilon, model, is_in_observed_state):
    # is_in_observed_state = (step <= 1000)
    # is_in_observed_state = (step <= NUM_OBSERVABLE_STEPS)
    rand_choice_is_under_epsilon_threshold = (np.random.rand() <= epsilon)
    if rand_choice_is_under_epsilon_threshold or is_in_observed_state:
    #if is_in_observed_state:
        return random.randrange(ACTION_OPTION_COUNT)
    else:
        q_value = model.predict([history, np.ones(ACTION_SIZE).reshape(1, ACTION_SIZE)])
        #q_value = model.predict([history, get_one_hot_encoded_action_mask()()])
    # Offset for 0 indexing of one-hot encoding array location of value
    return np.argmax(q_value[0]) + 1

def update_epsilon(total_steps, epsilon):
    training = (total_steps > NUM_OBSERVABLE_STEPS)
    epsilon_declining = epsilon > 0.1
    if epsilon_declining and training:
        epsilon -= EPSILON_DECAY
    return epsilon

def find_state_and_history(observed_state, history):
    next_state = preprocess(observed_state)
    # next_state = preprocess(observed_state)
    next_state = np.reshape([next_state], (1, 84, 84, 1))
    next_history = np.append(next_state, history)
    return (next_state, next_history)

def update_game_lifecycle(game_lives, info):
    game_dead = game_lives > info['ale.lives']
    game_lives = info['ale.lives']
    return (game_dead, game_lives)

def breakout_from_memory(memory):
    training_batch = random.sample(memory, BATCH_SIZE)
    history = init_batch_matrix()
    next_history = init_batch_matrix()
    action, reward, dead = [], [], []

    # Memory is stored in: indices 0 = history, 1 = action, 2 = reward, 3 = next_history, 4 = dead
    for index, val in enumerate(training_batch):
        history[index] = val[0]
        next_history[index] = val[3]
        action.append(val[1])
        reward.append(val[2])
        dead.append(val[4])
        
    return (history, next_history, action, reward, dead)

def get_one_hot_encoding(targets, nb_classes):
    # clip targets to range within 0 and 2 to make sure they are within possibilities.
    targets = np.clip(targets, 0, 2)
    # array for hot mapping each action
    # classes: 3, targets range from 0 to 2.  
    # The reshape -1 signifies that data is massaged into a dimension that is unknown.
    return np.eye(nb_classes)[np.array(targets).reshape(-1)]

def huber_loss(a, b):
    error = K.abs(a - b)
    quadratic_term = K.clip(error, 0.0, 1.0)
    linear_term = error - quadratic_term
    loss = K.mean(0.5 * K.square(quadratic_term) + linear_term)
    return loss


### Debugging Method
It is not necessary to run this but the training is so long it is useful to check in on how it is performing.
Some iterative logging function should be run in case the model quits while the programmer is sleeping or something.

In [71]:
def maybe_log_stuff(model, total_steps, player_game, score, loss, step, memory, file_writer):
    # debug.
    if player_game % 10 == 0:
        print('game: {}, total steps: {}, score: {}', (player_game, total_steps, score))
        
    # Record model state iteratively since run time is long.    
    if player_game % 1000 == 0 or (player_game + 1) == NUM_TURNS:
        now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
        file_name = "training_{}.h5".format(now)
        model_path = os.path.join(TRAIN_DIR, file_name)
        model.save(model_path)

    # Update File for storing model.
    loss_summary = tf.Summary(
        value=[tf.Summary.Value(tag="loss", simple_value=loss / float(step))])
    file_writer.add_summary(loss_summary, global_step=player_game)

    score_summary = tf.Summary(
        value=[tf.Summary.Value(tag="score", simple_value=score)])
    file_writer.add_summary(score_summary, global_step=player_game)


### Deep Q-Network Methods

In [72]:
def train_memory_batch(memory, model, log_dir):
    q_s_a = np.zeros((BATCH_SIZE,))
    history, next_history, action, reward, dead = breakout_from_memory(memory)

    actions_mask = np.ones((BATCH_SIZE, ACTION_OPTION_COUNT))
    # catj: predict for each action since mask is all 1s.
    next_Q_values = model.predict([next_history, actions_mask])

    # like Q Learning, get maximum Q value at s'
    # But from target model
    for i in range(BATCH_SIZE):
        if dead[i]:
            q_s_a[i] = reward[i]
        else:
            # Q(s, a) = r + gamma * max(Q(s', a'))
            # train model for future reward.
            q_s_a[i] = reward[i] + GAMMA * np.amax(next_Q_values[i])

    # Get an action for each possible reward.
    action_one_hot = get_one_hot_encoding(action, ACTION_OPTION_COUNT)
    # action mask on reward array.  mult each action by reward.
    # this will be the encoding of each action with an updated Q(s, a)
    rewards_one_hot = action_one_hot * q_s_a[:, None]
    
    # training data is on [history, action_one_hot], classifier is rewards
    # in predictions we will take highest reward and map to action using one hot technique.
    h = model.fit(
        [history, action_one_hot], rewards_one_hot, epochs=1,
        batch_size=BATCH_SIZE, verbose=0)

    return h.history['loss'][0]

# This function parents the deep Q Network if the model has enough memory for batch training.
def deep_q_iteration_training(memory, total_steps, model_clone, model):
    log_dir = None
    loss = train_memory_batch(memory, model, log_dir)
    if total_steps % MODEL_WEIGHTS_REFRESH_THRESOLD == 0:
        # Weights on the model clone get piped through so they only get updated as often as 
        # the treshold dictates the cycle update them.
        model_clone.set_weights(model.get_weights())
    return (model_clone, loss)

### Training Method

In [73]:
# # Mostly this function keeps track of system states, memory, and flags
# # It provides the opportunity to create logs for debugging
# # Most importantly it takes an action and updates a score.
# # It runs training on the model if all observation has been done.  This is Deep Q Learning.
def train():
    # Initialize global states.
    env, memory, epsilon, total_steps, player_games = init_config()
    # Initialize storage for collecting data on model performance.
    total_score = 0
    avg_game_scores = []
    # Get a copy of the cnn model with the architecture defined in a separate function.
    model = cnn_model()
    # Initialize file writer for logging. 
    
    # Initialize file writer.  
    # This is just used for logging and storing the model iteratively to preserve work.
    file_writer = init_file_writer_to_local_dir()
    
    # The main model gets used in the Q learning training, and based on updated weights, 
    # then also updates the model clone.  
    # Targeted Network update.
    model_clone = init_model_clone(model)
    
    # This is just a loop to cover the range of the global number of games played.
    # The player games number is kept visible to the program for logging purposes.
    while player_games < NUM_TURNS:
        # Define global game states.
        game_done, player_dead, game_step, game_score, game_lives, game_loss = init_game_config()
        # Reset the environment at the beginning of each game.
        observe = env.reset()

        # Prefill the start state, 4 frames.
        for _ in range(random.randint(1, INIT_NO_OP_STEPS)):
            observe, _, _, _ = env.step(1)
        history = init_history(observe)

        while not game_done:
            is_in_observed_state = (total_steps <= NUM_OBSERVABLE_STEPS)
            is_in_training_state = not is_in_observed_state
            
            # Epsilon has to decay a tiny bit with each iteration in the annealing method.
            epsilon = update_epsilon(total_steps, epsilon)

            # Get an action
            action = get_action(history, epsilon, model_clone, is_in_observed_state)

            # Take a step in the game
            observed_state, reward, game_done, info = env.step(action)
            
            # Update score based on agent action.
            # Move reward to the poles of 1 or -1 per the deep mind paper's suggestion
            game_reward = np.clip(reward, -1., 1.)
            game_score += game_reward
            
            # Preprocess state data and merge it with history.
            next_state = preprocess(observe)
            next_state = np.reshape([next_state], (1, 84, 84, 1))
            next_history = np.append(next_state, history[:, :, :, :3], axis=3)
            
            player_dead, game_lives = update_game_lifecycle(game_lives, info)
            
            # Update memory.
            memory.append((history, action, reward, next_history, player_dead))
            
            # Deep Q learning begins if the observational state is complete.
            # When the model has sufficiently recorded enough memory for training, start batch training.
            if is_in_training_state:
                model_clone, model_loss = deep_q_iteration_training(memory, total_steps, model_clone, model)
                game_loss += model_loss
                
            if not player_dead:
                # Update history to include the state if the agent didn't die.
                history = next_history
            
            # Update counts and state flags.
            player_dead = False
            # These are used more of less for logging and aren't too important to the system.
            total_steps += 1
            game_step += 1
            
            if game_done:
                maybe_log_stuff(model, total_steps, player_games, game_score, game_loss, game_step, memory, file_writer)
                total_score += game_score
                player_games += 1
                # update average game score log
                if total_steps % 100 == 0:
                    avg_game_score = total_score/100
                    avg_game_scores = avg_game_scores + avg_game_score
                    total_score = 0
    file_writer.close()
    return avg_game_scores


### Test Method

In [74]:
def test():
    env = gym.make('BreakoutDeterministic-v4')
    player_games = 0
    episode_number = 0
    epsilon = 0.001
    global_step = NUM_OBSERVABLE_STEPS + 1
    model = load_model(RESTORE_FILE_PATH, custom_objects={'huber_loss': huber_loss})

    while player_games < NUM_TURNS:
        # init variables
        game_done, game_dead, game_step, game_score, game_lives, game_loss = init_game_config()
        
        observe = env.reset()

        # Copy in initial states to amount to initial four frame history
        observe, _, _, _ = env.step(1)
        history = init_history(observe)
        while not done:
            if RENDER:
                env.render()
                time.sleep(0.01)
                
            is_in_observed_state = (global_step <= NUM_OBSERVABLE_STEPS)
            # Get action.
            action = get_action(history, epsilon, global_step, model)
            
            # Update epsilon.
            epsilon = update_epsilon(player_games, epsilon)

            # Take a step in the game.
            observed_state, reward, done, info = env.step(action)
            
            # Preprocess state to reduce image size, grayscale, and merge it with the history.
            # next history is the result of data augmentation and merge.
            next_state, next_history = find_state_and_history(observed_state)

            game_dead, game_lives = update_game_lifecycle(game_lives, info)

            # move reward to the poles of 1 or -1 per the deep mind paper's suggestion
            game_reward = np.clip(reward, -1., 1.)

            game_score += game_reward

            if not game_dead:
                # Update history to include the state if the agent didn't die.
                history = next_history

            # Update counts and state flags.
            dead = False
            # This is used more of less for logging and aren't too important to the system.
            game_step += 1
            
            if game_done:
                player_games += 1
                print('Games Played: ', player_games)
                print('Score: ', game_score)


## Implementation

### Imports

In [68]:
%%capture
import gym
import random
import numpy as np
import tensorflow as tf

from keras import layers
from keras.models import Model

from collections import deque
from keras.optimizers import RMSprop
from keras import backend as K
from skimage.color import rgb2gray
from skimage.transform import resize
from datetime import datetime
import os.path
import time
from keras.models import load_model
from keras.models import clone_model
from keras.callbacks import TensorBoard

import matplotlib.pyplot as plt

### Constants

In [69]:
TRAIN_DIR = 'openai_breakout_training_storage'
LOGS_FILE_PATH = '/Users/catherinejohnson/Downloads/Project Announcement-20201115/openai_breakout_training_storage/training_20201128034747.h5'
# suggested by Deep Mind Paper
NUM_TURNS = 1000
#NUM_TURNS = 100000
# suggested by Deep Mind Paper
NUM_OBSERVABLE_STEPS = 500
# NUM_OBSERVABLE_STEPS = 50000
MODEL_WEIGHTS_REFRESH_THRESOLD = 100
# MODEL_WEIGHTS_REFRESH_THRESOLD = 10000
# suggested by Deep Mind Paper
INIT_NO_OP_STEPS = 10
# INIT_NO_OP_STEPS = 30
REGULATION_SCALE = 0.01
# suggested by Deep Mind Paper
BATCH_SIZE = 32
LEARNING_RATE = 0.00025
GAMMA = 0.99
# suggested by Deep Mind Paper
ATARI_IMAGE_SHAPE = (84, 84, 4)
EPSILON_DECAY = ((1.0 - 0.1) / 1000000)
# suggested by Deep Mind Paper
ATARI_IMAGE_SHAPE = (84, 84, 4)
ACTION_OPTION_COUNT = 3

# suggested by Deep Mind Paper
LAYER_1_SIZE = 16
LAYER_1_FILTER = (8, 8)
LAYER_1_STRIDES = (4, 4)
LAYER_2_SIZE = 32
LAYER_2_FILTER = (4, 4)
LAYER_2_STRIDES = (2, 2)
ACTIVATION_FUNCTION = 'relu'

In [70]:
avg_game_scores_sesh = train()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image-shape-framework (InputLay (None, 84, 84, 4)    0                                            
__________________________________________________________________________________________________
normalize-images (Lambda)       (None, 84, 84, 4)    0           image-shape-framework[0][0]      
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 20, 20, 16)   4112        normalize-images[0][0]           
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 9, 9, 32)     8224        conv2d_17[0][0]                  
____________________________________________________________________________________________

NameError: name 'ACTION_SIZE' is not defined

In [ ]:
plt.plot(avg_game_scores_sesh)
print(avg_game_scores_sesh)
plt.plot([i for i, v enumerate(avg_game_scores_sesh)])
plt.title('plot')
plt.ylabel('avg scores')
plt.xlabel('time')
# plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# manually update file in RESTORE_FILE_PATH for testing.
test()